In [30]:
import tensorflow as tf
import numpy as np
import collections
import random
import math
tf.set_random_seed(777)

In [3]:
test1 = tf.constant([[1,2,3],[4,5,6],[7,8,9]])
test2 = tf.constant([0,1])
test3 = tf.nn.embedding_lookup(test1,test2)
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)
print(sess.run(test3))

[[1 2 3]
 [4 5 6]]


In [4]:
f = open("../text8", 'r', encoding='UTF8')
line = f.readline().split()
f.close()

In [5]:
dictionary = {}
count = {}
vocabulary = []
data = []
reverse_dictionary = {}

In [6]:
# dictionary 생성
dic_cnt = 0
for i in line :
    if i not in dictionary.keys() :
        dictionary[i] = dic_cnt
        dic_cnt += 1
    if i in count.keys() : 
        count[i] += 1
    else :
        count[i] = 1
    vocabulary.append(i)
    data.append(dictionary[i])
print(len(dictionary))

253854


In [7]:
for i in dictionary.keys() :
    reverse_dictionary[dictionary[i]] = i

In [8]:
print(len(vocabulary))

17005207


In [9]:
print(vocabulary[0], data[0])

anarchism 0


In [10]:
max_common = max(count.values())
for i in count.keys() :
    if count[i] == max_common :
        max_common_word = i

In [11]:
print("Most common word : ", max_common_word)

Most common word :  the


In [15]:
print(vocabulary[:10], data[:10])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against'] [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


##### batch와 label 생성

In [16]:
def generate_batch(batch_size, num_skips, skip_window, data_index) :
    
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    
    batch_cnt = 0
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    imsi_batch = np.ndarray(shape=(skip_window*2), dtype=np.int32)
    imsi_labels = np.ndarray(shape=(skip_window*2, 1), dtype=np.int32)
    while batch_cnt < batch_size :
        batch_pnt = data_index + skip_window
        for i in range(0,skip_window) :
            imsi_batch[i] = data[batch_pnt]
            imsi_labels[i,0] = data[batch_pnt-i-1]
        for i in range(0,skip_window) :
            imsi_batch[skip_window+i] = data[batch_pnt]
            imsi_labels[skip_window+i,0] = data[batch_pnt+i+1]
        
        skips = random.sample(range(0, 2*skip_window), num_skips)
        for i in skips :
            batch[batch_cnt] = imsi_batch[i]
            labels[batch_cnt] = imsi_labels[i]
            batch_cnt += 1
        data_index += 1
        if data_index == len(vocabulary) :
            data_index = 0
    return batch, labels, data_index

In [21]:
batch_size = 8
num_skips = 2
skip_window = 1
data_index = 0

In [19]:
batch_inputs, batch_labels, data_index = generate_batch(batch_size, num_skips, skip_window, data_index)

In [22]:
print(vocabulary[:10])
print(data[:10])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [23]:
for i in range(batch_size):
    print(batch_inputs[i], reverse_dictionary[batch_inputs[i]], '->', batch_labels[i, 0],
          reverse_dictionary[batch_labels[i, 0]])

1 originated -> 2 as
1 originated -> 0 anarchism
2 as -> 3 a
2 as -> 1 originated
3 a -> 2 as
3 a -> 4 term
4 term -> 3 a
4 term -> 5 of


In [27]:
# hyper-parameter 생성
batch_size = 128
vocabulary_size = len(vocabulary)
embedding_size = 128 
skip_window = 1 
num_skips = 2  
num_sampled = 64  

In [35]:
valid_size = 16  
valid_window = 100  
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

In [36]:
# minibatch 설정
train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

In [28]:
# embeddings 설정
embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

In [31]:
# projection 설정
nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],
                                stddev=1.0 / math.sqrt(embedding_size)))
nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

In [32]:
# loss 설정
loss = tf.reduce_mean(
          tf.nn.nce_loss(
              weights=nce_weights,
              biases=nce_biases,
              labels=train_labels,
              inputs=embed,
              num_sampled=num_sampled,
              num_classes=vocabulary_size))

<tf.Tensor 'loss:0' shape=() dtype=string>

In [33]:
optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

In [37]:
# minibatch와 전채 embedding 사이의 cosine 유사도 계산
# 해당 데이터와 가장 유사도 높은 단어를 끄집어 내 보려고
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

In [38]:
init = tf.global_variables_initializer()

In [39]:
num_steps = 1000001

In [ ]:
average_loss, data_index = 0, 0
with tf.Session() as sess :
    sess.run(tf.global_variables_initializer())
    for step in range(num_steps):
        batch_inputs, batch_labels, data_index = generate_batch(data, batch_size, num_skips, skip_window, data_index)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val
    if step % 2000 == 0:
        if step > 0:
            average_loss /= 2000
        print('Average loss at step {} : {}'.format(step, average_loss))
        average_loss = 0
    if step % 10000 == 0:
        sim = similarity.eval()         # (16, 50000)
        for i in range(valid_size):
            valid_word = reverse_dictionary[valid_examples[i]]
            print("Nearest to %s :"%valid_word, end = " ")
            top_k = 8
            nearest = (-sim[i, :]).argsort()[1:top_k + 1]
            for k in xrange(top_k):
                close_word = reverse_dictionary[nearest[k]]
                print("%s,"%close_word, end=" ")
        